<a href="https://colab.research.google.com/github/Kamohelo01/github-slideshow/blob/main/Europeancalulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Test/BinomialOptions.csv")
df.head()

,OptionID,Date,ExpiryDate,UnderlyingPrice,StrikePrice,ImpliedVolatility,PutCall,RiskFreeInterestRate
0,1,2021/10/04,2021/12/15,100.00,105,5%,C,6%
1,2,2021/10/04,2021/12/15,140.00,165,25%,C,6%
2,3,2021/10/04,2021/12/15,15.00,10,10%,P,6%
3,4,2021/10/04,2021/12/15,132.75,100,3%,P,6%
4,5,2021/10/04,2021/12/15,92.00,97,7%,C,6%


In [ ]:
df.columns

Index(['OptionID', 'Date', 'ExpiryDate', 'UnderlyingPrice', 'StrikePrice',
       'ImpliedVolatility', 'PutCall', 'RiskFreeInterestRate', 'percentage',
       'n', 't', 'q'],
      dtype='object')

In [ ]:
# Initialise parameters
S0 = UnderlyingPrice # initial stock price /1/
K = StrikePrice # strike price /2/
T = ExpiryDate - Date # time to maturity in years
r = RiskFreeInterestRate # annual risk-free rate
N = 5 # number of timesteps
# u =  # up-factor in binomial models
# d = 1/u # ensure recombining tree
sigma = ImpliedVolatility #Annualised stock price volatility
opttype = PullCall # Option Type 'C' or 'P'

NameError: ignored

Binomial Tree Slow

In [ ]:
def CRR_method(K,T,S0,r,N,sigma,opttype='PullCall'):
    #precomute constants
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    #initials asset prices at maturity - Time spend N
    S = np.zeros(N+1)
    S[0] = S0*d**N
    for j in range(1,N+1):
      S[j] = S[j-1]*u/d

    # initialise option values at maturity///
    C = np.zeros(N+1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    for j in range(0,N+1):
      C[j] = max(0,S[j]-K)

    # step backwars thourgh tree
    for i in np.arange(N,0,-1):
      for j in range(0,i):
        C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )

    return C[0]

CRR_method(K,T,S0,r,N,sigma,opttype='C')

TypeError: ignored

Binomial Tree Fast

In [ ]:

def binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C'):
    #precomute constants
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    #initials asset prices at maturity - Time spend N
    C = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))

    # initialise option values at maturity
    C = np.maximum( C - K, np.zeros(N+1) )

    # step backwars thourgh tree
    for i in np.arange(N,0,-1):
        C = disc * ( q * C[1:i+1] + (1-q) * C[0:i] )

    return C[0]
